# Fine tune distilbert to perform Text classification 

This notebook is intended to train `text-classification` models based on `distilbert base uncased` model. To do so we are using [Transformers 🤗🤗](https://huggingface.co/docs/transformers/index).

### Considerations
- The dataset must have column "text" where all the input questions are setted
- An `S3 Instance` is required to correctly store the model

#### Install required libs   📥📥

In [1]:
!pip install transformers datasets evaluate accelerate  mlflow tf-keras seaborn optimum[openvino,nncf,exporters] psutil pynvml -q

ERROR: Could not find a version that satisfies the requirement pynvml-q (from versions: none)
ERROR: No matching distribution found for pynvml-q

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Dataset manipulation & env preparation

In [2]:
#Define your input column and your output column
input_column_name="text"
output_column_name="sentiment"

In [3]:
import pandas as pd
import os
labeled_dataset = "datasets/dataset.csv"

# Assuming the file is in the current working directory
df = pd.read_csv(labeled_dataset)

# Display the first few rows of the dataframe
print(df.shape)
df.head()

(1999, 2)


,sentiment,text
0,spam,r\nj\nd\nz\np\nq
1,conversational,What are the steps outlined in the Development...
2,irrelevant,"I hope you have a wonderful day, po."
3,irrelevant,The Hudson River flows through New York.
4,spam,tZ8@L7x*K


## Give a name to your model and version  🧙‍♂️🧙‍♂️

This process is crucial mainly because a `text-classification` model can be intended for a huge amount of approaches

In [4]:
model_name = "intents-copa"
experiment = "showcases"
base_model_name = 'distilbert-base-uncased'
run_name = "V1 Intents for copa model"
test_size = 0.2

### Model manipulation 

In [5]:
# Read json files label2id &  id2label
import json
import os
# Opening JSON file
file_label2id = open('datasets/label2id.json')
file_id2label = open('datasets/id2label.json')

label2id = json.load(file_label2id)
id2label=json.load(file_id2label)
print(f"The label2id json loaded correctly: {label2id}")
print(f"The id2label json loaded correctly: {id2label}")

The label2id json loaded correctly: {'spam': 0, 'conversational': 1, 'irrelevant': 2}
The id2label json loaded correctly: {'0': 'spam', '1': 'conversational', '2': 'irrelevant'}


In [6]:
# Add the new 'label' column to the dataframe by mapping values from the 'category' column
df['label'] = df[output_column_name].replace(label2id)
df.head(3)

,sentiment,text,label
0,spam,r\nj\nd\nz\np\nq,0
1,conversational,What are the steps outlined in the Development...,1
2,irrelevant,"I hope you have a wonderful day, po.",2


In [7]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=test_size)

In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [10]:
def preprocess_function(examples):
    return tokenizer(examples[input_column_name], truncation=True)

In [11]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1599 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=len(label2id),id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

2024-08-12 20:03:49.806789: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-12 20:03:49.809857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 20:03:49.813623: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-12 20:03:49.825435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-12 20:03:49.844407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [15]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    conf_matrix = confusion_matrix(labels, predictions)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    
    # Save the confusion matrix plot
    conf_matrix_filepath = "confusion_matrix.png"
    plt.savefig(conf_matrix_filepath)
    plt.close()
    return metric.compute(predictions=predictions, references=labels)

In [16]:
#Mlflow
import mlflow

os.environ["MFLOW_EXPERIMENT_NAME"]=experiment

## Training

### Run name in Mlflow
Your must set a run name in mlflow in order to identify this run.

In [17]:
## hyperparameters 
lr=2e-5
train_batch_size= 8
eval_batch_size=8
epochs=2
decay=0.01
eval_strategy="epoch"
log_strategy="epoch"

In [18]:
training_args = TrainingArguments(
    hub_model_id=model_name,
    output_dir=run_name,
    learning_rate=lr,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    num_train_epochs=epochs,
    weight_decay=decay,
    evaluation_strategy=eval_strategy,
    logging_strategy=log_strategy,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=tokenized_test
)

/opt/app-root/lib64/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
import mlflow

mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(experiment)
mlflow.enable_system_metrics_logging()

In [20]:
### Fine tune model

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024/08/12 20:04:52 WARNING 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.205200,0.028636,0.995000
2,0.007500,0.017361,0.995000


2024/08/12 20:13:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run V1 Intents for copa model at: http://mlflow-tracking.mlflow.svc.cluster.local/#/experiments/15/runs/234201c0709b4a089409fc4c57e81847.
2024/08/12 20:13:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow-tracking.mlflow.svc.cluster.local/#/experiments/15.
2024/08/12 20:13:19 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/08/12 20:13:19 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


TrainOutput(global_step=400, training_loss=0.10633912652730942, metrics={'train_runtime': 505.834, 'train_samples_per_second': 6.322, 'train_steps_per_second': 0.791, 'total_flos': 38829681240156.0, 'train_loss': 0.10633912652730942, 'epoch': 2.0})

### Batch size per epoch

So if you have a batch size of 20 then 

total_dataset/batch_size = n

n represents the total amount of batches per epoch

### How many times does my model going to be trained?

n*epochs

In [21]:
## Save pytorch 
trainer.save_model(model_name)

In [22]:
experiment = mlflow.get_experiment_by_name(experiment)
filter_string = f"tags.mlflow.runName = '{run_name}'"
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    filter_string=filter_string
)

# Extract the run_id from the DataFrame
if not runs.empty:
    previous_run_id = runs.iloc[0]['run_id']
    print(f"Run ID: {previous_run_id}")
else:
    print("No run found with the specified name.")

Run ID: 234201c0709b4a089409fc4c57e81847


In [23]:
import mlflow.data
from mlflow.data.pandas_dataset import PandasDataset
from optimum.onnxruntime import ORTModelForSequenceClassification
from pathlib import Path
import onnx

In [24]:
def uploadModel(run_id:str):
    train_dataset: PandasDataset = mlflow.data.from_pandas(df_train, source="Label Studio")
    test_dataset: PandasDataset = mlflow.data.from_pandas(df_test, source="Label Studio")
    with mlflow.start_run(run_id=previous_run_id) as run:
        ORTModelForSequenceClassification.from_pretrained(model_name,export=True).save_pretrained(f"{model_name}_onnx")
        tmp_dir = Path(f"{model_name}_onnx")
        mlflow.log_artifacts(tmp_dir, artifact_path=model_name)
        mlflow.log_artifact("confusion_matrix.png",artifact_path=model_name)
        tokenizer.save_pretrained(f"{model_name}_onnx")
        onnx_model = onnx.load_model(f"{model_name}_onnx/model.onnx")
        model_info = mlflow.onnx.log_model(onnx_model,model_name,registered_model_name=model_name)
        mlflow.log_input(train_dataset, context="training")
        mlflow.log_input(test_dataset,context="validation")
        mlflow.end_run()

In [25]:
uploadModel(previous_run_id)

/opt/app-root/lib64/python3.9/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'Label Studio'. Exception: 
  return _dataset_source_registry.resolve(
/opt/app-root/lib64/python3.9/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/opt/app-root/lib64/python3.9/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'Label Studio'. Exception: 
  return _dataset_source_registry.resolve(
/opt/app-root/lib64/python3.9/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/app-root/lib64/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'intents-copa' already exists. Creating a new version of this model...
2024/08/12 20:14:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: intents-copa, version 2
Created version '2' of model 'intents-copa'.
2024/08/12 20:14:35 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
/opt/app-root/lib64/python

In [26]:
mlflow.end_run()

In [27]:
## Delete directories in Jupyter Notebook
import shutil

# Remove the local model directory
shutil.rmtree(model_name)
shutil.rmtree(run_name)
os.remove(labeled_dataset)
shutil.rmtree(f"{model_name}_onnx")
os.remove("datasets/label2id.json")
os.remove("datasets/id2label.json")
os.remove("./confusion_matrix.png")

---